In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from war_functions.pecota_tables import *
from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *
from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *
from odds_and_other_projections import *

In [4]:
bets = pd.read_csv('past_bets/base/bets_20220728.csv', index_col = 0)

In [5]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Pittsburgh Pirates,Philadelphia Phillies,0.355337,0.644663,160.0,-190.0,0.384615,0.655172,-0.029279,-0.010509,0.000000,0.000000,0.000000,0.000000
1,New York Yankees,Kansas City Royals,0.717283,0.282717,-235.0,194.0,0.701493,0.340136,0.015791,-0.057419,0.005290,0.000000,594.025598,0.000000
2,Toronto Blue Jays,Detroit Tigers,0.715937,0.284063,-225.0,188.0,0.692308,0.347222,0.023630,-0.063159,0.007680,0.000000,862.377602,0.000000
3,Boston Red Sox,Cleveland Guardians,0.385315,0.614685,100.0,-118.0,0.500000,0.541284,-0.114685,0.073400,0.000000,0.016001,0.000000,1796.856321
4,Houston Astros,Seattle Mariners,0.576172,0.423828,-148.0,126.0,0.596774,0.442478,-0.020602,-0.018650,0.000000,0.000000,0.000000,0.000000
5,Colorado Rockies,Los Angeles Dodgers,0.370594,0.629406,180.0,-215.0,0.357143,0.682540,0.013451,-0.053134,0.002092,0.000000,234.970490,0.000000
6,San Francisco Giants,Chicago Cubs,0.621081,0.378919,-146.0,124.0,0.593496,0.446429,0.027586,-0.067510,0.006786,0.000000,762.035904,0.000000


In [6]:
bets.to_csv('past_bets/base/bets_20220728.csv')

In [17]:
# Fixing error

# Necessary parameters
current_year = dt.date.today().year
today = str(dt.date.today()).replace('-', '')
today_date = dt.date.today()
yesterday = today_date - dt.timedelta(days=1)
yesterday_string = str(yesterday)
yesterday_string = yesterday_string.replace('-', '')
team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians' : 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

# Getting yesterdays results from CBS
link = 'https://www.cbssports.com/mlb/scoreboard/' + yesterday_string + '/'
tables = pd.read_html(link)
results_table = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Winner'])
team_regex = r'\D*'
for table in tables:
    if list(table.columns) == ['Unnamed: 0', 'R', 'H', 'E']:

        # Getting team names
        team_away_string = table.iloc[0,0]
        team_away_list = re.findall(team_regex, team_away_string)
        team_away = team_away_list[0]
        
        team_home_string = table.iloc[1,0]
        team_home_list = re.findall(team_regex, team_home_string)
        team_home = team_home_list[0]
        
        # Getting score and determining winner
        runs_away = table.iloc[0,1]
        runs_home = table.iloc[1,1]
        if runs_away>runs_home:
            winner = team_away
        else:
            winner = team_home
        
        # Appending to results table
        series = pd.Series([team_home, team_away, winner], index = results_table.columns)
        results_table = results_table.append(series, ignore_index = True)        
    else:
        continue
for column in list(results_table.columns):
    results_table[column] = results_table[column].apply(lambda x: team_map[x])

results_table

,Home_Team,Away_Team,Winner
0,New York Yankees,Tampa Bay Rays,New York Yankees
1,Chicago Cubs,San Francisco Giants,San Francisco Giants
2,Oakland Athletics,Chicago White Sox,Chicago White Sox
3,Kansas City Royals,Detroit Tigers,Detroit Tigers
4,Baltimore Orioles,Boston Red Sox,Boston Red Sox
5,Philadelphia Phillies,Washington Nationals,Philadelphia Phillies
6,Miami Marlins,New York Mets,New York Mets
7,Texas Rangers,Toronto Blue Jays,Toronto Blue Jays
8,Milwaukee Brewers,Cincinnati Reds,Milwaukee Brewers
9,Minnesota Twins,Cleveland Guardians,Cleveland Guardians


In [6]:
tables

[     Unnamed: 0   R   H  E
 0     Rays78-59   3   7  0
 1  Yankees84-56  10  15  1,
                                          0
 0  W: J. Taillon (13-4)L: C. Kluber (10-8),
            0      1
 0  Box Score  Recap,
     Unnamed: 0  R   H  E
 0  Giants66-73  5  11  1
 1    Cubs58-81  2   7  0,
                                                    0
 0  W: L. Webb (13-8)L: M. Stroman (3-7)S: C. Dova...,
            0      1
 0  Box Score  Recap,
        Unnamed: 0   R   H  E
 0  White Sox72-68  10  20  2
 1  Athletics50-90   2   3  0,
                                       0
 0  W: L. Lynn (6-5)L: A. Martinez (4-4),
            0      1
 0  Box Score  Recap,
     Unnamed: 0  R   H  E
 0  Tigers54-85  8  11  1
 1  Royals56-84  4   4  1,
                                         0
 0  W: M. Manning (2-2)L: J. Heasley (3-8),
            0      1
 0  Box Score  Recap,
      Unnamed: 0   R   H  E
 0  Red Sox68-72  17  21  1
 1  Orioles73-66   4   8  0,
                                        0